In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import tflearn
import tensorflow as tf
import random

Instructions for updating:
Colocations handled automatically by placer.


In [2]:
import json
with open('../intents/intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/reynadoerwald/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
#classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

3113 documents
711 classes ['hypertensive disease', 'diabetes', 'depression mental', 'depressive disorder', 'coronary arteriosclerosis', 'coronary heart disease', 'pneumonia', 'failure heart congestive', 'accident cerebrovascular', 'asthma', 'myocardial infarction', 'hypercholesterolemia', 'infection', 'infection urinary tract', 'anemia', 'chronic obstructive airway disease', 'dementia', 'insufficiency renal', 'confusion', 'degenerative polyarthritis', 'hypothyroidism', 'anxiety state', 'malignant neoplasms', 'primary malignant neoplasm', 'acquired immuno-deficiency syndrome', 'HIV', 'hiv infections', 'cellulitis', 'gastroesophageal reflux disease', 'septicemia', 'systemic infection', 'sepsis (invertebrate)', 'deep vein thrombosis', 'dehydration', 'neoplasm', 'embolism pulmonary', 'epilepsy', 'cardiomyopathy', 'chronic kidney failure', 'carcinoma', 'hepatitis ', 'peripheral vascular disease', 'psychotic disorder', 'hyperlipidemia', 'bipolar disorder', 'obesity', 'ischemia', 'cirrhosis'

In [5]:
training = []
output = []

output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

In [6]:
tf.reset_default_graph()

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

#model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.load('model.tflearn')


Training Step: 389999  | total loss: 2.63457 | time: 1.214s
| Adam | epoch: 1000 | loss: 2.63457 - acc: 0.1453 -- iter: 3112/3113
Training Step: 390000  | total loss: 2.58006 | time: 1.218s
| Adam | epoch: 1000 | loss: 2.58006 - acc: 0.1558 -- iter: 3113/3113
--
INFO:tensorflow:/Users/reynadoerwald/Desktop/Honours/medical-chatbot/scripts/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [7]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)

    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)

    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [8]:
p = bow("Nausea", words)
print (p)
print (classes)

[0 0 0 ... 0 0 0]
['hypertensive disease', 'diabetes', 'depression mental', 'depressive disorder', 'coronary arteriosclerosis', 'coronary heart disease', 'pneumonia', 'failure heart congestive', 'accident cerebrovascular', 'asthma', 'myocardial infarction', 'hypercholesterolemia', 'infection', 'infection urinary tract', 'anemia', 'chronic obstructive airway disease', 'dementia', 'insufficiency renal', 'confusion', 'degenerative polyarthritis', 'hypothyroidism', 'anxiety state', 'malignant neoplasms', 'primary malignant neoplasm', 'acquired immuno-deficiency syndrome', 'HIV', 'hiv infections', 'cellulitis', 'gastroesophageal reflux disease', 'septicemia', 'systemic infection', 'sepsis (invertebrate)', 'deep vein thrombosis', 'dehydration', 'neoplasm', 'embolism pulmonary', 'epilepsy', 'cardiomyopathy', 'chronic kidney failure', 'carcinoma', 'hepatitis ', 'peripheral vascular disease', 'psychotic disorder', 'hyperlipidemia', 'bipolar disorder', 'obesity', 'ischemia', 'cirrhosis', 'exanth

In [9]:
print(model.predict([p]))

[[1.59034319e-02 2.01281086e-02 1.04542360e-05 8.91364743e-06
  1.99816888e-04 2.44152558e-04 3.90895593e-06 8.71677116e-07
  1.12332818e-04 1.38876501e-07 2.82936567e-03 7.23956339e-03
  2.70819437e-04 7.48095976e-04 4.83217882e-03 3.94927582e-07
  1.01716502e-03 4.70169540e-03 7.05646176e-04 2.17558397e-03
  6.69185724e-03 1.43016677e-03 7.20801472e-04 3.20310995e-04
  5.04149148e-06 5.49246033e-06 4.84682050e-06 2.35436572e-04
  6.10065982e-02 1.10776722e-03 1.02013373e-03 1.15575688e-03
  5.30442003e-06 2.49361414e-02 4.53404710e-03 1.05077959e-06
  8.23914206e-07 1.84399643e-04 1.63379163e-02 9.37934685e-03
  8.87040244e-07 2.09866630e-04 5.95968368e-06 5.37950452e-03
  2.67087762e-06 3.96258052e-04 6.55252778e-04 6.92622329e-04
  2.71301542e-04 4.67881386e-04 5.75236743e-04 2.17091455e-03
  3.56884208e-04 2.16714830e-06 3.60269610e-06 8.56021943e-04
  1.40852382e-04 2.51168356e-04 1.18525003e-08 7.27546811e-02
  6.32913652e-05 8.81205196e-04 1.73077844e-02 3.57471115e-04
  4.4854

In [10]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [18]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.01
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [24]:
classify("my son is now nineteen months old and doesnt say any words at all. he jabbers a lot but you can't understand any of it. he seems very normal otherwise. he plays and laughs and smiles. i've read things about how autistic children don't hold eye contact. he can and does. we have a seven year old son as well and i don't remember it taking this long for him to talk. should i be concerned that he may be autistic? i am a nervous wreck over this.")

[('autism and language', 0.28227443),
 ('behaviour children', 0.18239072),
 ('autistic children symptoms', 0.07042944),
 ('autism symptoms children', 0.06623266),
 ('behaviour problems', 0.061518814),
 ('pediatrician', 0.05182751),
 ('autism symptoms of', 0.048046842),
 ('autism communication', 0.045206804),
 ('autism', 0.03999158),
 ('hypertension pulmonary', 0.033874124),
 ('symptoms', 0.01594817)]

In [26]:
response("my son is now nineteen months old and doesnt say any words at all. he jabbers a lot but you can't understand any of it. he seems very normal otherwise. he plays and laughs and smiles. i've read things about how autistic children don't hold eye contact. he can and does. we have a seven year old son as well and i don't remember it taking this long for him to talk. should i be concerned that he may be autistic? i am a nervous wreck over this.")

autism is a medical condition in which there are problems with communication and social interaction. this is a rather serious condition to be diagnosed without a complex and complete examination. the child might lack eye contact as you mentioned. the babbling in these children might be accompanied with unusual gestures repeating othersâ words whereas the words are less frequently associated with appropriate gestures. repetitive behaviour might be one of the symptoms that are most easy to recognize. it might include: compulsive behaviour ritualistic behaviour stereotypy or even self-injury. as for the slow improvement of the speech please consider that many children just need more time to talk and to be understood. different conditions might be causing this situation. sometimes the child might need help from a specialist for speech and language therapy whom you are advised to visit. s/he might be able to detect if there are any physiological problems with the speech-related organs (th

In [27]:
classify('I think my child has autism.')

[('mitral valve insufficiency', 0.15444137),
 ('infection', 0.114869796),
 ('stenosis aortic valve', 0.05699378),
 ('osteomyelitis', 0.041458104),
 ('deep vein thrombosis', 0.03224081),
 ('cardiomyopathy', 0.022025887),
 ('failure heart', 0.021085443),
 ('thrombocytopaenia', 0.0196005),
 ('septicemia', 0.019353615),
 ('sepsis (invertebrate)', 0.018094353),
 ('systemic infection', 0.017544307),
 ('failure kidney', 0.015003482),
 ('cellulitis', 0.012999724),
 ('symptoms', 0.012986791),
 ('treatment', 0.012921259),
 ('lymphoma', 0.0120896045),
 ('exanthema', 0.01017379)]

In [29]:
response('I think my child has autism.')

We will be with you as soon as we can


In [30]:
response('Goodbye, see you later')

It looks like you may have primary carcinoma of the liver cells


In [33]:
classify("how can a patient quit smoking?")

[('addiction', 0.24473102),
 ('stop smoking', 0.22128315),
 ('hypertension pulmonary', 0.18145563),
 ('vision and eye disorders', 0.028218998),
 ('septicemia', 0.022935428),
 ('taking supplements', 0.020347072),
 ('sepsis (invertebrate)', 0.019646993),
 ('dependence', 0.019476287),
 ('systemic infection', 0.019469732),
 ('neurological disorders', 0.017649073),
 ('embolism pulmonary', 0.014275271),
 ('vitamins', 0.0113377115),
 ('hyperbilirubinemia', 0.010254316)]

In [34]:
response('how can a patient quit smoking?')

stopping smoking is about will power and being steadfast. you can stop safely by having bupropion or nicotine patch cover initially in consult with a doctor. contact an addiction clinic near you. wishing you best of health thanks


In [54]:
context

{}

In [55]:
response('today')

In [56]:
classify('today')

[]

In [57]:
response("Hi there!", show_details=True)

In [58]:
response('today')
classify('today')

[]

In [59]:
response("thanks, your great")